# Plots for resampled cleaned chartevents chunks

Script for creating time series plots, which allow to compare vital parameter values series before and after resampling.

In [ ]:
import pandas as pd
import pyarrow as pa

# Read from parquet files
chartevents_before = pd.read_parquet('../data/chartevents_clean_values_and_thresholds_with_chunkid_65.parquet', engine='pyarrow')
chartevents_resampled_median = pd.read_parquet('../data/chartevents_clean_values_and_thresholds_with_chunkid_65_resampled_median.parquet', engine='pyarrow')
chartevents_resampled_mean = pd.read_parquet('../data/chartevents_clean_values_and_thresholds_with_chunkid_65_resampled_mean.parquet', engine='pyarrow')

In [ ]:
chunkid = '226799.0_220045.0_2153-07-31 08:10:00' # Selection of a single chunk
# Chunk IDs of other examples:
# '260223.0_220045.0_2156-07-22 06:49:00'
# '203164.0_220045.0_2178-02-08 03:00:00'
# '296490.0_220045.0_2192-09-26 23:51:00'
# '226799.0_220045.0_2153-07-31 08:10:00'

itemid = 220045

before = chartevents_before[
    (chartevents_before.CHUNK_ID_FILLED_TH == chunkid) & (chartevents_before.ITEMID == itemid)][
    ['CHARTTIME','VALUENUM_CLEAN']].sort_values(by=['CHARTTIME'])

after_median = chartevents_resampled_median[(chartevents_resampled_median.CHUNK_ID_FILLED_TH == chunkid) & (chartevents_resampled_median.ITEMID == itemid)][
    ['CHARTTIME','VALUENUM_CLEAN']].sort_values(by=['CHARTTIME']).rename(columns={"VALUENUM_CLEAN": "MEDIAN"}).set_index('CHARTTIME')

after_mean = chartevents_resampled_mean[(chartevents_resampled_mean.CHUNK_ID_FILLED_TH == chunkid) & (chartevents_resampled_mean.ITEMID == itemid)][
    ['CHARTTIME','VALUENUM_CLEAN']].sort_values(by=['CHARTTIME']).rename(columns={"VALUENUM_CLEAN": "MEAN"}).set_index('CHARTTIME')

In [ ]:
merged_resampling_methods = pd.concat([after_median,after_mean],axis=1)

# Melt for seaborn plot
after = pd.melt(merged_resampling_methods.reset_index(),'CHARTTIME')
after = after.sort_values(by=['CHARTTIME']).rename(columns={"variable": "RESAMPLING_METHOD", "value": "VALUENUM_CLEAN"}).reset_index(drop=True)

In [ ]:
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np
import datetime as dt

sns.set_style("whitegrid")

fig, axs = plt.subplots(
    2,
    1,
    figsize = (15, 8),
    sharex = True,
    dpi = 72 # e.g. 72 for screen, 300 for print
    )
fig.suptitle(str("CHUNK ID: "+str(chunkid)), fontweight='bold', color= 'black', fontsize=14, y=0.95)
# fig.subplots_adjust(hspace = 0.6)

ylimits = (min(before.VALUENUM_CLEAN)-5, max(before.VALUENUM_CLEAN)+5)

ax = sns.lineplot(
    ax = axs[0],
    data = before,
    x = "CHARTTIME",
    y = "VALUENUM_CLEAN",
    drawstyle = 'steps-post', # Interpolate missing values by using the last available value
    marker="o",
    markersize = 5,
    dashes = False,
    palette = [sns.color_palette("colorblind")[0]]
        )
axs[0].set_ylim(ylimits)
axs[0].set_title("Heart Rate - Before Resampling", fontweight='bold', color= 'black', fontsize=12, y=1.025)
axs[0].set_xlabel("Time", fontsize=12, labelpad=15)
axs[0].set_ylabel("Beats per minute", fontsize=12, labelpad=15)
axs[0].tick_params(axis="x", rotation=90)

ax = sns.lineplot(
    ax = axs[1],
    data = after,
    x = "CHARTTIME",
    y = "VALUENUM_CLEAN",
    hue = "RESAMPLING_METHOD",
    drawstyle = 'steps-post', # Interpolate missing values by using the last available value
    marker="o",
    markersize = 5,
    dashes = False,
    palette = [sns.color_palette("colorblind")[0],sns.color_palette("colorblind")[1]]
        )
axs[1].legend(loc='center left', bbox_to_anchor=(1, 0.5))
axs[1].set_ylim(ylimits)
axs[1].set_title("Heart Rate - After Resampling", fontweight='bold', color= 'black', fontsize=12, y=1.025)
axs[1].set_xlabel("Time", fontsize=12, labelpad=15)
axs[1].set_ylabel("Beats per minute", fontsize=12, labelpad=15)
axs[1].tick_params(axis="x", rotation=90)

path = '../plots_week07/'
timestamp = dt.datetime.today().strftime('%Y-%m-%dT%H-%M-%S')
plt.savefig(str(path + '_resampling_plot_' + timestamp + '.png'), dpi=300, bbox_inches='tight')

# plt.show(fig)